In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import torch
import os
import sys
import yaml

SCRIPT_DIR = os.getcwd()
sys.path.append(os.path.dirname(SCRIPT_DIR))
print(SCRIPT_DIR) 

from modules.data_module import *
from modules.evaluation_module import *
from modules.forecast_module import *
from modules.observation_module import *
from modules.visualisation_module import *
from misc.helpers import *

PATH_TO_PLOTS = '../plots/'
path_to_results = '../results/'

#EX_CONFIG = load_config(config_path = '../../configs/smosmania_st.yaml')
EX_CONFIG = load_config(config_path = '../../configs/smosmania_st.yaml')
EX_STATION = "Condom"

# List of paths to the .nc files
# '/perm/dadf/HSAF_validation/in_situ_data/pre_processed_data/ismn_nc/soil_TERENO_ISMN_2022.nc'
#dir_to_data = "/perm/dadf/HSAF_validation/in_situ_data/pre_processed_data/ismn_nc"
station_name = 'soil_SMOSMANIA_ISMN_2008.nc'

all_stations = ['CabrieresdAvignon', 'Condom', 'CreondArmagnac', 'LaGrandCombe',
       'Lahas', 'LezignanCorbieres', 'Mazan-Abbaye', 'Mejannes-le-Clap',
       'Montaut', 'Mouthoumet', 'Narbonne', 'PeyrusseGrande', 'Pezenas-old',
       'Sabres', 'SaintFelixdeLauragais', 'Savenes', 'Urgons', 'Villevielle']
use_stations = ['Condom', 'Villevielle', 'LaGrandCombe', 'Narbonne']

Station = ObservationModule(network = EX_CONFIG['network'], 
                                station = EX_STATION ,
                                variable = EX_CONFIG['variable'],
                                depth=EX_CONFIG['depth']) # Initialise the Observation Module with the default Station (Gevenich)
    

Station.load_station(years = EX_CONFIG['years']) # Load two years of station data for lookback slicing
Station.load_forcing() # Load forcing for matching data base with station data
Station.match_station_with_forcing() # Match the station with clostest grid cell and extract the index of the grid cell
Station.process_station_data() # specify path_to_plots, if you want to visualise
station_data = Station.slice_station_data(lookback=0,t_0=EX_CONFIG['initial_time'])

DOY_VECTOR = Station.doy_vector

In [4]:
stations_dict = {}
for station in use_stations:
    with open(f"../results/SMOSMANIA_{station}_2022_st_ensemble.yaml", 'r') as f:
        layers = yaml.load(f, Loader=yaml.UnsafeLoader)
    stations_dict[station] = layers

forecast_dict = {}
for station in use_stations[:7]:
    with open(f"../results/SMOSMANIA_{station}_2022_st_ensemble_fc.yaml", 'r') as f:
        layers_fc = yaml.load(f, Loader=yaml.UnsafeLoader)
    forecast_dict[station] = layers_fc

In [6]:
PlotStations = VisualisationMany(
                 network = EX_CONFIG["network"], 
                 station = "all", 
                 variable = EX_CONFIG["variable"], 
                 maximum_leadtime = 672, 
                 score = EX_CONFIG["score"],
                 doy_vector = DOY_VECTOR,
                 evaluation = "ens", 
                 path_to_plots = PATH_TO_PLOTS
)

PlotStations.assemble_scores(stations_dict)

layers = {}

for layer, scores in enumerate([PlotStations.scores_l1, PlotStations.scores_l2, PlotStations.scores_l3]):

    layers[f"layer{layer}"] = {}

    layers[f"layer{layer}"]["h_upper"] = np.argmax((1 - scores["skill_scores_upper"]) < 0)
    layers[f"layer{layer}"]["h_mean"] = np.argmax((1 - scores["skill_scores_mean"]) < 0)
    layers[f"layer{layer}"]["h_lower"] = np.argmax((1 - scores["skill_scores_lower"]) < 0)

In [7]:
print(layers)

{'layer0': {'h_upper': np.int64(1), 'h_mean': np.int64(4), 'h_lower': np.int64(8)}, 'layer1': {'h_upper': np.int64(1), 'h_mean': np.int64(10), 'h_lower': np.int64(57)}, 'layer2': {'h_upper': np.int64(0), 'h_mean': np.int64(15), 'h_lower': np.int64(300)}}
